In [ ]:
import os
import cv2
import random
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import RMSprop,Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.vgg16 import preprocess_input,VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
%matplotlib inline

In [ ]:
fldr = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/agepred/code/Users/sanjaysriram991/utkcropped'

In [ ]:
files = os.listdir(fldr)
plt.imshow(cv2.imread(files[0]))

In [ ]:
ages=[]
#genders=[]
images=[]

for fle in files:
    if fle[0].isdigit():
        age = int(fle.split('_')[0])
        ages.append(age)
        #gender = int(fle.split('_')[1])
        #genders.append(gender)
        total = fldr+'/'+fle
        print(total)
        image = cv2.imread(total)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image,(224,224))
        image = preprocess_input(image)
        images.append(image)
        if len(images) == 15000:
            break

In [ ]:
print(len(images))
print(len(ages))
#print(len(genders))

In [ ]:
images_f=np.array(images)
#genders_f=np.array(genders)
ages_f=np.array(ages)

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
gender = ['Male', 'Female']
values=[4372,5047]
ax.bar(gender,values)
plt.show()

In [ ]:
values, counts = np.unique(ages_f, return_counts=True)
print(counts)

In [ ]:
plt.plot(counts)
plt.xlabel('ages')
plt.ylabel('distribution')
plt.show()

In [ ]:
'''labels=[]

i=0
while i < len(images):
    label=[]
    label.append([ages[i]])
    label.append([genders[i]])
    labels.append(label)
    i+=1
labels'''

In [ ]:
images_f_2=images_f/255

In [ ]:
ages_f=np.array(ages) #labels instead of age
len(ages_f)

In [ ]:
images_f.shape

In [ ]:
X_train, X_test, Y_train, Y_test= train_test_split(images_f, ages_f,test_size=0.25) # change ages_f to labels_f

In [ ]:
#Y_train_2=[Y_train[:,1],Y_train[:,0]]
#Y_test_2=[Y_test[:,1],Y_test[:,0]]

In [ ]:
# Huber loss or L1 smooth loss seems to work out best for regression with CNNs

In [ ]:
pre_trained_model = VGG16(input_shape=(224,224,3),
                                   include_top = False,
                                   weights = 'imagenet')
for layer in pre_trained_model.layers:
    layer.trainable = False

x = layers.Flatten()(pre_trained_model.output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(1, activation='linear')(x)           

model = Model(pre_trained_model.input, x) 
    
model.compile(optimizer = 'Adam', 
              loss = tf.keras.losses.Huber(), 
              metrics = ['acc']) 
    
model.summary()

In [ ]:
aug = ImageDataGenerator(rotation_range=40, width_shift_range=0.2, 
                         height_shift_range=0.2, horizontal_flip=True)

In [ ]:
hist = model.fit(aug.flow(X_train, Y_train, batch_size=64),
                 validation_data=(X_test, Y_test),
                 steps_per_epoch= len(X_train)//64,
                 epochs=100, verbose=1)
# use Y_train_2 and Y_test_2

In [ ]:
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV

In [ ]:
def create_model():

    pre_trained_model = VGG16(input_shape=(96,96,3),
                                   include_top = False,
                                   weights = 'imagenet')
    for layer in pre_trained_model.layers:
        layer.trainable = False

    x = layers.Flatten()(pre_trained_model.output)
    x = layers.Dense(1024, activation='relu')(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(1, activation='linear')(x)           

    model = Model(pre_trained_model.input, x) 
    
    model.compile(optimizer = 'Adam', 
                  loss = tf.keras.losses.Huber(), 
                  metrics = ['acc']) 
    
    return model

In [ ]:
models = KerasRegressor(build_fn=create_model,verbose=1)

In [ ]:
batch_size=[64,128]
epochs = [50,100]
#optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']

In [ ]:
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=models, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(images_f, ages_f)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
model.evaluate(X_test,Y_test) #use Y_test_2

In [ ]:
pred = model.predict(X_test)

In [ ]:
pred[1]

In [ ]:
def test_image(ind,images_f,images_f_2,model):
    plt.imshow(images_f[ind])
    image_test=images_f_2[ind]
    pred_1=model.predict(np.array([image_test]))
    #print(pred_1)
    #sex_f=['Male','Female']
    age=int(np.round(pred_1[1][0]))
    #sex=int(np.round(pred_1[0][0]))
    print("Predicted Age: "+ str(age))
    #print("Predicted Sex: "+ sex_f[sex])

In [ ]:
test_image(57,images_f,images_f_2,model)

In [ ]:
test_image(137,images_f,images_f_2,model)

In [ ]:
test_image(502,images_f,images_f_2,model)

In [ ]:
test_image(24,images_f,images_f_2,model)